In [4]:
import os
from users.base_user import UserSim
from users.llm_user import LLMUserSim
from dialogue_sim import DialogueSimulator
import pe_modules
import users
import dataloaders
import llms
import jinja2
import argparse
import yaml
import json

In [7]:
#Read data
with open('data/synth_data.json') as f:
    shape_data = json.load(f)['items']

In [8]:
shape_data

[{'id': 0,
  'name': '0',
  'description': ['Colour: Red, Size: Large, Shape: Circle']},
 {'id': 1,
  'name': '1',
  'description': ['Colour: Red, Size: Large, Shape: Triangle']},
 {'id': 2,
  'name': '2',
  'description': ['Colour: Red, Size: Small, Shape: Circle']},
 {'id': 3,
  'name': '3',
  'description': ['Colour: Red, Size: Small, Shape: Triangle']},
 {'id': 4,
  'name': '4',
  'description': ['Colour: Blue, Size: Large, Shape: Circle']},
 {'id': 5,
  'name': '5',
  'description': ['Colour: Blue, Size: Large, Shape: Triangle']},
 {'id': 6,
  'name': '6',
  'description': ['Colour: Blue, Size: Small, Shape: Triangle']},
 {'id': 7,
  'name': '7',
  'description': ['Colour: Blue, Size: Small, Shape: Triangle']}]